# 🥈 Silver Layer: Diabetes 30-Day Readmission Project
This notebook processes the Bronze layer data to create a cleaned, feature-rich Silver dataset for downstream analytics and modeling. Key steps include:
* Reading raw data from the Bronze Delta table
* Cleaning and transforming features
* Engineering the 30-day readmission target variable
* Validating schema and class distribution

The Silver dataset is now ready for advanced analysis and machine learning.

In [0]:
# STEP 1: Read data from Bronze layer
# Load the raw diabetes readmission data from the Bronze Delta table.
silver_df = spark.table("diabetes_readmissions.bronze_diabetes_readmission")

In [0]:
# STEP 2: Replace '?' with null values
# Standardize missing values by replacing '?' with None (null).
silver_df = silver_df.replace("?", None)

In [0]:
# STEP 3: Drop identifier columns
# Remove unique identifiers that are not useful for modeling.
silver_df = silver_df.drop("encounter_id", "patient_nbr")

In [0]:
# STEP 4: Create 30-day readmission target variable
# Engineer the binary target column: 1 if readmitted within 30 days, else 0.
from pyspark.sql.functions import when, col

silver_df = silver_df.withColumn(
    "readmitted_30",
    when(col("readmitted") == "<30", 1).otherwise(0)
)

In [0]:
# STEP 6: Check class distribution
# Display the distribution of the target variable to assess class imbalance.
display(silver_df.groupBy("readmitted_30").count())
# Helps us understand class imbalance (very common in healthcare problems)

readmitted_30,count
0,90409
1,11357


In [0]:
# STEP 7: Check schema
# Print the schema to verify data types and structure.
silver_df.printSchema()

root
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- admission_type_id: integer (nullable = true)
 |-- discharge_disposition_id: integer (nullable = true)
 |-- admission_source_id: integer (nullable = true)
 |-- time_in_hospital: integer (nullable = true)
 |-- payer_code: string (nullable = true)
 |-- medical_specialty: string (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- number_outpatient: integer (nullable = true)
 |-- number_emergency: integer (nullable = true)
 |-- number_inpatient: integer (nullable = true)
 |-- diag_1: string (nullable = true)
 |-- diag_2: string (nullable = true)
 |-- diag_3: string (nullable = true)
 |-- number_diagnoses: integer (nullable = true)
 |-- max_glu_serum: string (nullable = true)
 |-- A1Cresult: string (nullable = true)
 |-- m

In [0]:
# STEP 8: Preview cleaned Silver data
# Show a sample of the cleaned Silver dataset for validation.
silver_df.select(
    "race",
    "gender",
    "age",
    "time_in_hospital",
    "num_medications",
    "number_inpatient",
    "A1Cresult",
    "insulin",
    "readmitted_30"
).show(10, truncate=False)

+---------------+------+--------+----------------+---------------+----------------+---------+-------+-------------+
|race           |gender|age     |time_in_hospital|num_medications|number_inpatient|A1Cresult|insulin|readmitted_30|
+---------------+------+--------+----------------+---------------+----------------+---------+-------+-------------+
|Caucasian      |Female|[0-10)  |1               |1              |0               |None     |No     |0            |
|Caucasian      |Female|[10-20) |3               |18             |0               |None     |Up     |0            |
|AfricanAmerican|Female|[20-30) |2               |13             |1               |None     |No     |0            |
|Caucasian      |Male  |[30-40) |2               |16             |0               |None     |Up     |0            |
|Caucasian      |Male  |[40-50) |1               |8              |0               |None     |Steady |0            |
|Caucasian      |Male  |[50-60) |3               |16             |0     

In [0]:
# STEP 9: Save Silver DataFrame as Delta Table
# Persist the cleaned Silver DataFrame as a Delta table for downstream use.
silver_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("diabetes_readmissions.silver_diabetes_readmission")
# This saves the cleaned Silver DataFrame as a Delta table for downstream analytics and modeling.

The Silver dataset contains a large number of clinical and medication-related features. When displayed in the notebook console, the output appears wrapped due to width constraints, but the data remains fully tabular and structured, which is verified through schema inspection and column validation.
